# Imports & paths

### This block imports all the required libraries and give paths

In [1]:
#  Imports & paths --
import os, pandas as pd
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
ROOT = r"D:\\MedicalAI-Assistant"
DATA_DIR = os.path.join(ROOT, 'data')
ART_DIR = os.path.join(ROOT, 'artifacts')
os.makedirs(ART_DIR, exist_ok=True)

C:\Users\mahin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
W0824 14:15:24.401000 8304 torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


#  Load reports CSV robustly 

In [2]:
csv_path = os.path.join(DATA_DIR, 'Reports.csv')
texts = []
if os.path.exists(csv_path):
 df = pd.read_csv(csv_path)
 # Prefer a column named like a report
 for col in ['report_text', 'Report', 'report', 'text', 'notes']:
  if col in df.columns:
   texts = df[col].dropna().astype(str).tolist()
  break
 if not texts:
 # fallback: first object dtype column
  for col in df.columns:
   if df[col].dtype == 'object':
    texts = df[col].dropna().astype(str).tolist()
   break
 print('Loaded reports:', len(texts))

Loaded reports: 3931


# Build summarizer (T5-small) and test 

In [3]:
model_name = 't5-small'
tok = AutoTokenizer.from_pretrained(model_name)
mod = AutoModelForSeq2SeqLM.from_pretrained(model_name)
SUM = pipeline('summarization', model=mod, tokenizer=tok)

sample = texts[0] if texts else (
 "This is a sample clinical report describing patient symptoms and findings including fever, cough, and dyspnea."
)

print(SUM(sample, max_length=120, min_length=20, do_sample=False)[0]['summary_text'])

Device set to use cpu
Your max_length is set to 120, but your input_length is only 46. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=23)
Both `max_new_tokens` (=256) and `max_length`(=120) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


allergy / immunology, allergic rhinitis, allergies, asthma, nasal sprays, nasal, erythematous, allegra, sprays .
